In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # torch_dtype=torch.float32,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    # attn_implementation="sdpa",
    attn_implementation="flash_attention_2",
)
# model = model.eval()
model.requires_grad_(False)

from datasets import load_dataset

dataset = load_dataset(
    # "dim/hendrycks_math_train_12k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

dataset = dataset["train"].train_test_split(
    # test_size=250,
    test_size=350,
    # test_size=999,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)

correct_dataset = []

for pos, item in enumerate(dataset):
    try:
        answer = dataset_answer_filter(item["answer"])
        model_answer = model_answer_filter(item["model_answer"])
        # print(answer, model_answer)
        # break
        if is_equiv(answer, model_answer):
            correct_dataset.append(item)
    except:
        pass

print(len(dataset), len(correct_dataset), len(correct_dataset) / len(dataset))

# correct_dataset = correct_dataset[:30]
len(correct_dataset)

'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
209 185 0.8851674641148325


185

In [2]:
correct_dataset[0]

{'problem': 'If $A=2+i$, $O=-4$, $P=-i$, and $S=2+4i$, find $A-O+P+S$.',
 'solution': 'Adding real parts and imaginary parts separately, we have $(2-(-4)+0+2)+(1+0-1+4)i=\\boxed{8+4i}$.',
 'answer': '8+4i',
 'subject': 'Algebra',
 'level': 3,
 'unique_id': 'train/algebra/3.json',
 'model_answer': "Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2 + i, O is -4, P is -i, and S is 2 + 4i. So, replacing each term:\n\n= (2 +

In [ ]:
import torch
import gc
from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
from tqdm.notebook import tqdm
from tqdm import tqdm as text_tqdm
from hidden_capacity_reasoning.utils import tokenize_single_turn

torch.manual_seed(0)
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

for dataset_pos in tqdm(range(1, len(correct_dataset))):
    tokenized_turn = tokenize_single_turn(
        question=base_prompt.format(question=correct_dataset[dataset_pos]["problem"]),
        answer=correct_dataset[dataset_pos]["model_answer"],
        tokenizer=tokenizer,
    )
    for key in tokenized_turn.keys():
        tokenized_turn[key] = torch.tensor(tokenized_turn[key])

    device = "cuda"

    content_compression_mask = tokenized_turn["content_compression_mask"]

    input_part_end = (content_compression_mask == 0).nonzero()[-3][0]
    # get only question part
    question_input_ids = (
        tokenized_turn["input_ids"][: int(input_part_end) + 1].unsqueeze(0).cuda()
    )
    print(tokenizer.decode(question_input_ids[-1]))

    question_embeds = model.get_input_embeddings()(question_input_ids)

    mem_tokens = 128 * 2
    max_len = 4096
    amount = (max_len - question_embeds.shape[1]) // mem_tokens

    compression_tensor_param = torch.nn.Parameter(
        torch.rand(
            mem_tokens,
            model.get_input_embeddings().weight.shape[1],
            device="cuda",
        ).unsqueeze(0),
        requires_grad=True,
    )

    repeat_amount = 4096 - question_embeds.shape[1] - 2

    optimizer = torch.optim.AdamW(
        [
            compression_tensor_param,
        ],
        lr=0.1,
    )
    compression_tensor = compression_tensor_param.repeat(1, amount, 1)

    input_embeds = torch.cat(
        [
            question_embeds,
            compression_tensor,
        ],
        dim=1,
    )

    input_labels = tokenized_turn["input_ids"].to("cuda")

    input_part_end = (content_compression_mask == 1).nonzero()[-1][0] + 1
    input_part_end
    input_labels[:input_part_end][content_compression_mask[:input_part_end] == 1] = -100
    epoch_amount = 200
    dtype = model.dtype
    new_input_labels = torch.cat(
        [
            input_labels,
            torch.tensor(
                (input_embeds.shape[1] - input_labels.shape[0])
                * [tokenizer.eos_token_id],
                device=input_labels.device,
            ),
        ],
        # dim=1,
    )  # .unsqueeze(0)
    new_input_labels[new_input_labels == tokenizer.eos_token_id] = -100
    new_input_labels[input_labels.shape[0] - 1] = tokenizer.eos_token_id
    # with torch.autocast(device_type="cuda", dtype=dtype):
    model_answer = correct_dataset[dataset_pos]["model_answer"]
    for epoch in range(epoch_amount):
        compression_tensor = compression_tensor_param.repeat(1, amount, 1)
        input_embeds = torch.cat(
            [
                question_embeds,
                compression_tensor,
            ],
            dim=1,
        ).to(dtype)
        model_predicts = model(
            inputs_embeds=input_embeds,
            labels=new_input_labels,
        )

        prediction_str = tokenizer.decode(
            model_predicts.logits.argmax(-1)[-1][input_part_end + 2 :]
        )
        if "<｜end▁of▁sentence｜>" in prediction_str:
            prediction_str = prediction_str[
                : prediction_str.index("<｜end▁of▁sentence｜>")
            ]
            if prediction_str == model_answer:
                print("Correct")
                break

        compression_loss = model_predicts.loss
        compression_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(
        "compression_loss",
        compression_loss,
        epoch,
        len(tokenizer.encode(model_answer)),
    )
    if prediction_str != model_answer:
        print("Wrong")

    print("=====")
    print("=====")
    del compression_tensor_param
    del compression_tensor
    del optimizer
    del input_embeds
    del model_predicts
    del compression_loss
    gc.collect()
    gc.collect()
    gc.collect()
    torch.cuda.empty_cache()
    # break

  0%|          | 0/184 [00:00<?, ?it/s]

<｜begin▁of▁sentence｜><｜User｜>Problem: How many numbers are in the list $25, 26, 27, \ldots, 99, 100 ?$

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>

Correct
compression_loss tensor(0.0284, device='cuda:0', grad_fn=<NllLossBackward0>) 91 1767
=====
=====
<｜begin▁of▁sentence｜><｜User｜>Problem: The number of white balls and red balls in a jar is in the ratio of $3:2$. If there are 9 white balls, how many red balls are there?

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>

Correct
compression_loss tensor(0.0601, device='cuda:0', grad_fn=<NllLossBackward0>) 54 826
=====
=====
<｜begin▁of▁sentence｜><｜User｜>Problem: Bill travels the 400 miles from San Francisco to Los Angeles at 50 mph. Sam travels the same distance at 40 mph. How many more hours than Bill did it take Sam to travel the 400 miles?

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>

Correct
compress